# Locally Linear Embedding Method

The LLE algorithm is one of the many dimensionality reduction methods based on spectral theory. It attempts to create a low dimensional embedding such that vectors that are nearby in high dimension remain close in low dimensional embedding. The error function is minimized in such a way that the neighborhood around a vector does not change in a lower dimension. The LLE  algorithm uses conformal maps to solve the problem. The conformal or biholomorphic maps preserve the local angles between the vectors. It creates embedding solely based on neighborhood distances without using global distances. LLE assumes that data lies on a smooth manifold (i.e., it does not have holes) and each vector and its neighbors lie approximately on a locally linear patch on the manifold. The latter assumption gives us the freedom to express each vector as a weighted sum of its neighbors.   

LLE starts by building conformal maps of the original dataset and then replicates it in the lower dimensions. 

Suppose , we have $X_i$ where $i\;\in (1,...,n) \;\text{and}\; X_i\;\in R^D$. LLE begins by creating a neighborhood matrix. It assumes that dataset is large and well-sampled, i.e. for every vector we have enough vectors to create a K-nearest or $\epsilon$-ball neighborhood.   

Once the neighborhood matrix is defined, each vector can be reconstructed as a linear weighted sum of its neighbors. The cost function for reconstruction can be formulated as:
 
$\hspace{7em}\mathcal{E}(W) = \Sigma_{i}(X_i - \Sigma_{j\in N(i)}^k\; W_{ij}X_j)^2$  

LLE obtained its name based on the nature of these reconstructions. Since only neighbors participate in reconstruction, it is *local*. The reconstruction has been achieved by linear coefficients or weights, hence *linear*.  

The weights $W_{ij} $ that contributed to the reconstruction of $Y_i$  in higher dimensions should contribute to the reconstruction of the $Y_i$ in a lower dimension. Based on this idea,  the cost (error) function is defined as    

$\hspace{7em}\Phi(Y) = \Sigma_{i}(Y_i - \Sigma_{j}W_{ij}Y_j)^2$  

where $Y$ is the embedding, we need to find. 

The difference between the two error functions defined above is that the weights $W_{ij}$ are fixed in the latter case while in the former they are variables. The embedding error does not depend on the $X$ (original dataset) and is a function of geometric information encoded by the weight matrix. With few suitable constraints, error function can be solved using eigenvalue decomposition of $(I-W)^T(I-W)$ ($I$ is an identity matrix), and a unique solution was obtained. The reconstruction weights of vectors $X_i$ and $X_j$ are independent of each other. The eigenvalue decomposition is a global operation which processes the information provided by all data. It presents the step where geometric information from the weight matrix has incorporated into the global structure of the embedding.


#### Neighbor search   

Neighbourhood can be created  through k-nearest neighbor (k-NN) or $\epsilon$-ball neighborhood approach.  

**K-nearest neighbor** - At this method, each vector was connected to its K-nearest vectors. By using this technique, we will always have K-neighbors for every vector. Since a vector selects K-vectors and some other vector may select it as a neighbor who is not in his neighborhood set. This situation generally arises in case of an isolated vector which selects faraway vectors as neighbors, while its neighbors can select neighborhood set from a smaller distance. It produces asymmetric neighborhood matrix.  


**$\epsilon$-ball neighbor** - Each vector $X_i$ selects every vector inside the ball with radius $\epsilon$ and centered at $X_i$ as its neighbors. This approach sometimes leads to vectors with no neighbors. It is hard to find the right $\epsilon$ since the smaller value will give many isolated vectors and higher value will have many neighbors for each vector. This approach is useful for approximating geodesic distances.

**Step 1: The seeking of neighbors in Python**


In [11]:
from sklearn.neighbors import NearestNeighbors 
from sklearn import datasets, neighbors
import numpy as np


# The implementation of K-nearest neighbor (K-NN) search method


def KNNAlgorithm(X, K, t = 2.0, dist_metric = "euclidean", algorithm = "ball_tree"):
    
    #n, p = X.shape
    
    knn = neighbors.NearestNeighbors(K+1, metric = dist_metric, algorithm=algorithm).fit(X)
    
    distances, nbors = knn.kneighbors(X)
    
    return(nbors[:,1:])

#### The computing of the reconstruction weights W

We will also assume that dataset is not too noisy so that we do not have significant outliers who distort the weights. 

LLE tries to reconstruct the vector $X_i$ as the weighted average of its neighbors. The reconstruction error is given by  

$\hspace{7em}\mathcal{E}(W) = \Sigma_{i}(X_i - \Sigma_{j\in N(i)}^k\; W_{ij}X_j)^2$ 

where 

$\hspace{7em}N(i)$ is neighbourhood set of $X_i$ and 

$\hspace{7em} \Sigma_{j}W_{ij}\; =\; 1.0\;$  


The matrix $W$ has a property that it is invariant to rescalings, rotations, and translations. The invariance to rotations and rescalings comes from error formulation while $sum_{row}(W)$ = 1 keeps $W_{ij}$ unchanged in case of translations. Using matrix algebra, we get a closed form solution to this problem. 

It is important to note that if the number of neighbors K is higher than original dimension *D*, we will not have the unique solution and some of the $W_{ij}$ might be zero. This issue can be handled by adding a regularization term penalizing the large weights.  

Steps to minimize $\mathcal{E}(W)$  

For $i$ in $1:n$;  
$\hspace{2em}$create a matrix $Z$ with all neighbors of $X_i$  
$\hspace{2em}$subtract $X_i$ from $Z$  
$\hspace{2em}$create the local covariance matrix $C = ZZ^T$  
$\hspace{2em}$Add a regularized term to avoid C being singular, $C$ $= C + reg*I$  
$\hspace{2em}$solve $CW = 1$ for $W$  
$\hspace{2em}$set $W_{ij} = 0$ if j is not a neighbor of i  
$\hspace{2em}$set $W = W/sum(W)$  



**Step 2: The computing of the reconstruction weights W in Python**

In [31]:
# The calculation of the reconstruction weights W

from scipy import linalg

def getReconstructionWeights(X, nbors, reg, K):
    
    N, D = X.shape
    
    Weights = np.zeros((N, N))
    
    for i in range(N):
        
        X_bors = X[nbors[i],:] - X[i] # subtract $X_i$ from $Z$, where Z=nbors[i] for all i  
        
        cov_nbors = np.dot(X_bors, X_bors.T) # The local covariance matrix $C = ZZ^T$ 
        
        
        trace = np.trace(cov_nbors) # The regularization term
        if trace >0 :
            R = reg*trace 
        else:
            R = reg
        
        # R is a regularized term to avoid C being singular
        
        cov_nbors.flat[::K+1] += R #C+=reg*I
        
        weights = linalg.solve(cov_nbors, np.ones(K).T, sym_pos=True) # Solve $CW = 1$ for $W 

        weights = weights/weights.sum()
        
        Weights[i, nbors[i]] = weights 
        
    return(Weights)

#### The calculation of the embedded data using the weights W

Now, come's the last step of the algorithm, i.e. computing the embedded data with the help of reconstruction weights. The error for the reconstruction of data in a lower dimension is given by  

$\hspace{7em}\Phi(Y) = \Sigma_{i}(Y_i - \Sigma_{j}W_{ij}Y_j)^2$  


Since, $\Sigma_{j}W_{ij} = 1.0$ we can write  

$\hspace{7em}\Phi(Y) = \Sigma_{i}(\Sigma_{j}W_{ij}(Y_i - Y_j)^2) = \Sigma_{i}(\Sigma_{j}(Y_i - Y_j)W_{ij}(Y_i - Y_j)^T) $   
$\hspace{9em}= \text{tr}\; Y^TMX$    

whre the matrix $M$ is given by $M$ = $(I-W)^T(I-W)$. We need to add few constraints to make sure we have stable solutions  

$\hspace{7em} \Sigma_i Y_i = 0$  
$\hspace{7em} YY^T = I\;\; and I$ is identity matrix  


Minimizing $\Phi(Y)$ with the constraints leads to unique solution in form of eigenvalue decomposition of $M$.

We need to select $2^{Nd}$ to $(K+1)^{th}$ smallest eigenvectors as the embedded data.





**Step 3: The computing of the embedded vectors by using the weights W in Python**


In [32]:
# The computing the bottom eigenvectors (embedding coordinates) by using weights W

from scipy.linalg import eigh

def getEmbeddedVectors(Weights, d):
    
    N, K = Weights.shape
    
    I = np.eye(N)
    
    m = (I-Weights)
    
    M = m.T.dot(m)
    
    eigvals, eigvecs = eigh(M, eigvals=(1, d), overwrite_a=True)
    
    ind = np.argsort(np.abs(eigvals))
    
    return(eigvecs[:, ind])

In [33]:
# The LLE Algorithm 

def lleAlgorithm(X, K, d, reg):
       
    nbors= KNNAlgorithm(X, K)
    
    Weights = getReconstructionWeights(X, nbors, reg, K)
    
    Y = getEmbeddedVectors(Weights, d)
    
    return(Y)

In [50]:
# The Main Program

import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
from ipywidgets import interact, interactive, fixed, interact_manual
#import ipywidgets as widgets

reg=0.001
d=2
K=10
n_points = 1000
X, color = datasets.samples_generator.make_s_curve(n_points, random_state=0)

N, D= X.shape # D is equal 3, and N=1000

Y = lleAlgorithm(X, K, d, reg)

test = [354, 520, 246, 134, 3, 983, 186, 436, 893, 921]
s = Y[test]

#fig = plt.figure(figsize=(10,8))
#plt.scatter(Y[:,0],Y[:,1],c=color, cmap = cm.get_cmap("Spectral"))
#plt.scatter(s[:,0],s[:,1], c="black")

def draw(K):
    fig = plt.figure(figsize=(10,8))
    reg=0.0001
    Y = lleAlgorithm(X, K, 2, reg)
    s = Y[test]
    plt.scatter(Y[:,0],Y[:,1],c=color, cmap = cm.get_cmap("Spectral"))
    plt.scatter(s[:,0],s[:,1], c="black")
    
interact(draw, K= widgets.IntSlider(min=10, max=100, value=10, step=10))

interactive(children=(IntSlider(value=10, description='K', min=10, step=10), Output()), _dom_classes=('widget-…

<function __main__.draw(K)>